# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,stanley,54.8680,-1.6985,17.50,63,40,4.63,GB,1690284710
1,1,margaret river,-33.9500,115.0667,13.78,80,89,5.29,AU,1690284704
2,2,waitangi,-43.9535,-176.5597,11.01,98,100,2.68,NZ,1690284711
3,3,aasiaat,68.7098,-52.8699,3.13,91,97,1.96,GL,1690284712
4,4,acajutla,13.5928,-89.8275,26.24,76,89,2.22,SV,1690284712


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,bethel,41.3712,-73.4140,24.16,88,0,1.54,US,1690284716
39,39,clarinda,40.7419,-95.0383,23.18,100,0,0.00,US,1690284725
76,76,merritt island,28.5392,-80.6720,26.66,86,0,2.06,US,1690284740
186,186,ocala,29.1872,-82.1401,23.99,89,0,1.54,US,1690284778
203,203,holualoa,19.6228,-155.9522,26.04,86,0,4.12,US,1690284785
244,244,sao vicente,-23.9631,-46.3919,24.04,100,0,0.00,BR,1690284802
261,261,naeso,35.2497,128.5200,25.60,94,0,0.00,KR,1690284809
273,273,laguna,38.4210,-121.4238,21.16,71,0,3.09,US,1690284814
284,284,fort bragg,35.1390,-79.0060,24.34,97,0,1.54,US,1690284818
288,288,palmas,-10.2128,-48.3603,25.93,47,0,2.06,BR,1690284820


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
15,bethel,US,41.3712,-73.4140,88,
39,clarinda,US,40.7419,-95.0383,100,
76,merritt island,US,28.5392,-80.6720,86,
186,ocala,US,29.1872,-82.1401,89,
203,holualoa,US,19.6228,-155.9522,86,
244,sao vicente,BR,-23.9631,-46.3919,100,
261,naeso,KR,35.2497,128.5200,94,
273,laguna,US,38.4210,-121.4238,71,
284,fort bragg,US,35.1390,-79.0060,97,
288,palmas,BR,-10.2128,-48.3603,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
clarinda - nearest hotel: Celebrity Inn
merritt island - nearest hotel: No hotel found
ocala - nearest hotel: Hilton Garden Inn
holualoa - nearest hotel: Kona Hotel
sao vicente - nearest hotel: Pousada Vitória
naeso - nearest hotel: 아리랑관광호텔
laguna - nearest hotel: Holiday Inn Express & Suites
fort bragg - nearest hotel: Airborne Inn Lodging
palmas - nearest hotel: Hotel Triângulo Mineiro
sinop - nearest hotel: No hotel found
belyy yar - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
yarmouth - nearest hotel: The Inn At Cape Cod
stephenville - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
pass christian - nearest hotel: Pearl Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
15,bethel,US,41.3712,-73.4140,88,Hampton Inn Danbury
39,clarinda,US,40.7419,-95.0383,100,Celebrity Inn
76,merritt island,US,28.5392,-80.6720,86,No hotel found
186,ocala,US,29.1872,-82.1401,89,Hilton Garden Inn
203,holualoa,US,19.6228,-155.9522,86,Kona Hotel
244,sao vicente,BR,-23.9631,-46.3919,100,Pousada Vitória
261,naeso,KR,35.2497,128.5200,94,아리랑관광호텔
273,laguna,US,38.4210,-121.4238,71,Holiday Inn Express & Suites
284,fort bragg,US,35.1390,-79.0060,97,Airborne Inn Lodging
288,palmas,BR,-10.2128,-48.3603,47,Hotel Triângulo Mineiro


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )




# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)